In [2]:
import pandas as pd
import numpy as np
import scipy 

ag = pd.read_excel("ag-202510.xlsx", sheet_name="Raw Data Report")

/home/randy/work/yume/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
ag[['影片播放到 25% 的次數', '影片播放到 50% 的次數', '影片播放到 75% 的次數', '影片播放到 95% 的次數', '影片播放到 100% 的次數']] = ag[['影片播放到 25% 的次數', '影片播放到 50% 的次數', '影片播放到 75% 的次數', '影片播放到 95% 的次數', '影片播放到 100% 的次數']].fillna(0)
ag['p25'] = (ag['影片播放到 25% 的次數'] / ag['曝光次數']) * 100
ag['p50'] = (ag['影片播放到 50% 的次數'] / ag['曝光次數']) * 100
ag['p75'] = (ag['影片播放到 75% 的次數'] / ag['曝光次數']) * 100
ag['p95'] = (ag['影片播放到 95% 的次數'] / ag['曝光次數']) * 100
ag['p100'] = (ag['影片播放到 100% 的次數'] / ag['曝光次數']) * 100

ag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10165 entries, 0 to 10164
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   行銷活動名稱          10165 non-null  object 
 1   天數              10165 non-null  object 
 2   廣告組合名稱          10165 non-null  object 
 3   廣告名稱            10165 non-null  object 
 4   年齡              10165 non-null  object 
 5   性別              10165 non-null  object 
 6   曝光次數            10165 non-null  int64  
 7   影片平均播放時間        9785 non-null   float64
 8   CTR（全部）         10164 non-null  float64
 9   CPM（每千次廣告曝光成本）  10164 non-null  float64
 10  觸及人數            10165 non-null  int64  
 11  連結點擊次數          4520 non-null   float64
 12  CPC（單次連結點擊成本）   4520 non-null   float64
 13  影片播放到 25% 的次數   10165 non-null  float64
 14  影片播放到 50% 的次數   10165 non-null  float64
 15  影片播放到 75% 的次數   10165 non-null  float64
 16  影片播放到 95% 的次數   10165 non-null  float64
 17  影片播放到 100% 的次數  10165 non-null 

In [ ]:
def decay_from_play_rates(play_rates, video_length_sec=None):
    """
    play_rates: list/array of 5 values (25, 50, 75, 95, 100% play rates) in PERCENT (e.g. 16.97)
    video_length_sec: float or None
    
    Returns: dict with k, half-life as fraction, and half-life in seconds (if length given)
    """
    PROGRESS = np.array([
        0.00,
        0.25, 
        0.50, 
        0.75, 
        0.95, 
        1.00
        ]) 
    # convert % → fraction
    # r = np.array(play_rates, dtype=float) / 100.0
    epsilon = 1e-9
    r = np.array(play_rates, dtype=float) / 100.0
    r = np.clip(r, epsilon, 1.0)  # Replace 0s with epsilon to avoid log(0)
    
    log_r = np.log(r)
    slope, intercept = np.polyfit(PROGRESS, log_r, 1)
    
    k = -slope                       # decay constant
    half_frac = np.log(2) / k        # half-life as fraction of video
    
    return {
        "k": np.round(k, 2),
        "half_life_fraction": np.round(half_frac, 2)
    }

def compute_decay_row(row):
    play_rates = [
        row['p0'],
        row['p25'],
        row['p50'],
        row['p75'],
        row['p95'],
        row['p100']
    ]
    out = decay_from_play_rates(play_rates, None)
    return pd.Series(out)

ag_decay = ag.groupby("廣告名稱")[['曝光次數', '影片播放到 25% 的次數', '影片播放到 50% 的次數', '影片播放到 75% 的次數', '影片播放到 95% 的次數', '影片播放到 100% 的次數']].sum().reset_index()
ag_decay['p0'] = 100.0
ag_decay['p25'] = (ag_decay['影片播放到 25% 的次數'] / ag_decay['曝光次數']) * 100
ag_decay['p50'] = (ag_decay['影片播放到 50% 的次數'] / ag_decay['曝光次數']) * 100
ag_decay['p75'] = (ag_decay['影片播放到 75% 的次數'] / ag_decay['曝光次數']) * 100
ag_decay['p95'] = (ag_decay['影片播放到 95% 的次數'] / ag_decay['曝光次數']) * 100
ag_decay['p100'] = (ag_decay['影片播放到 100% 的次數'] / ag_decay['曝光次數']) * 100

ag_decay[ag_decay[['p25', 'p50', 'p75', 'p95', 'p100']] < 0] = 0.0
ag_decay[['decay_k', 'half_life_fraction']] = ag_decay.apply(
    compute_decay_row, axis=1
)

ag_decay.sort_values("decay_k", ascending=False)

,廣告名稱,曝光次數,影片播放到 25% 的次數,影片播放到 50% 的次數,影片播放到 75% 的次數,影片播放到 95% 的次數,影片播放到 100% 的次數,p0,p25,p50,p75,p95,p100,decay_k,half_life_fraction
0,和司特_Alga3藻油_原物力,11,2.0,0.0,0.0,0.0,0.0,100.0,18.181818,0.000000,0.000000,0.000000,0.000000,22.80,0.03
12,和司特_KD_原物力2,1544,3.0,1.0,0.0,0.0,0.0,100.0,0.194301,0.064767,0.000000,0.000000,0.000000,22.21,0.03
14,和司特_KD_原物力_2C,39,1.0,0.0,0.0,0.0,0.0,100.0,2.564103,0.000000,0.000000,0.000000,0.000000,22.00,0.03
10,和司特_Alga3藻油_藥事報老大3_2C,1719,1.0,1.0,0.0,0.0,0.0,100.0,0.058173,0.058173,0.000000,0.000000,0.000000,21.70,0.03
11,和司特_KD_原物力,2,0.0,0.0,0.0,0.0,0.0,100.0,0.000000,0.000000,0.000000,0.000000,0.000000,15.01,0.05
3,和司特_Alga3藻油_原物力_2C,481,0.0,0.0,0.0,0.0,0.0,100.0,0.000000,0.000000,0.000000,0.000000,0.000000,15.01,0.05
46,和司特_Probiotical_尬天借膽8_2B,1,0.0,0.0,0.0,0.0,0.0,100.0,0.000000,0.000000,0.000000,0.000000,0.000000,15.01,0.05
54,和司特_Probiotical_藥事報老大4_2C,16,0.0,0.0,0.0,0.0,0.0,100.0,0.000000,0.000000,0.000000,0.000000,0.000000,15.01,0.05
2,和司特_Alga3藻油_原物力Barry_2C,420,46.0,4.0,2.0,1.0,1.0,100.0,10.952381,0.952381,0.476190,0.238095,0.238095,5.92,0.12
33,和司特_Naticol_藥事報老大9_2B,1778,30.0,11.0,7.0,3.0,3.0,100.0,1.687289,0.618673,0.393701,0.168729,0.168729,5.50,0.13


In [25]:
ag_decay.drop(ag_decay[(ag_decay['p25'] == 0) | (ag_decay['p50'] == 0) | (ag_decay['p75'] == 0) | (ag_decay['p95'] == 0) | (ag_decay['p100'] == 0)].index).sort_values("廣告名稱", ascending=False).to_excel("ag-decay-202510.xlsx", index=False)

In [71]:
ag['decay_k'] = np.where(ag['p25'] == 0, 0, ag['decay_k'])
ag['half_life_fraction'] = np.where(ag['p25'] == 0, 0, ag['half_life_fraction'])

In [72]:
ag.groupby("廣告名稱")[['p25', 'p50', 'p75', 'p95', 'p100', 'decay_k', 'half_life_fraction']].mean()

,p25,p50,p75,p95,p100,decay_k,half_life_fraction
廣告名稱,,,,,,,
和司特_Alga3藻油_原物力,11.904762,0.000000,0.000000,0.000000,0.000000,6.278553,9.013121e-03
和司特_Alga3藻油_原物力Barry,3.723838,0.497207,0.285427,0.113639,0.102296,6.121868,1.365666e+13
和司特_Alga3藻油_原物力Barry_2C,7.348179,2.545247,2.435065,2.380952,2.380952,6.912997,-inf
和司特_Alga3藻油_原物力_2C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
和司特_Alga3藻油_打電話,11.803365,3.650167,1.706204,0.748947,0.597860,9.714316,5.028455e+13
和司特_Alga3藻油_藥事報老大1,6.216261,2.190021,1.434517,1.095788,0.877115,7.894809,-inf
和司特_Alga3藻油_藥事報老大1_2C,6.693047,1.667660,1.153739,0.976434,0.847608,8.158577,1.368092e+14
和司特_Alga3藻油_藥事報老大2,14.174197,10.512102,7.029398,3.883477,3.713607,8.459912,-inf
和司特_Alga3藻油_藥事報老大2_2C,8.213677,4.855142,3.248001,1.750775,1.547130,6.610110,3.142676e+14


In [ ]:
17.113757, 4.582011, 3.063492, 0.698413, 0.698413 (decay k = 9.285681)
16.897437, 7.054135, 4.962602, 3.459695, 2.876937 (decay k = 10.744781)